In [77]:
##Packages
import inference
import os
import shutil
import numpy as np
from collections import Counter
import supervision as sv
from ultralytics import YOLO
import glob

In [78]:
def compute_iou(box1, box2):
    # Compute intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute area of intersection
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    # Compute area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Compute IoU
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou

def filter_detections(detection):
    boxes = detection[0].boxes.xyxy.tolist()
    confidences = detection[0].boxes.conf.tolist()
    class_ids = detection[0].boxes.cls.int().tolist()
    class_names = [detection[0].names[int(class_id)] for class_id in class_ids]

    
    num_boxes = len(boxes)
    if num_boxes < 2:
        return {
            'xyxy': boxes,
            'confidence': confidences,
            'class_id': class_ids,
            'class_name': class_names
        }
        
    to_keep = [True] * num_boxes

    for i in range(num_boxes):
        if not to_keep[i]:
            continue
        for j in range(i + 1, num_boxes):
            if compute_iou(boxes[i], boxes[j]) > 0.90:
                if confidences[i] < confidences[j]:
                    to_keep[i] = False
                else:
                    to_keep[j] = False

    # Filter the boxes based on the to_keep list
    filtered_boxes = [box for box, keep in zip(boxes, to_keep) if keep]
    filtered_confidences = [box for box, keep in zip(confidences, to_keep) if keep]
    filtered_class_ids = [box for box, keep in zip(class_ids, to_keep) if keep]

    #class_names = [detection[0].names[int(class_id)] for class_id in class_ids]
    filtered_class_names = [box for box, keep in zip(class_names, to_keep) if keep]
    
    return {
        #Don't need most of this
        'xyxy': filtered_boxes,
        'confidence': filtered_confidences,
        'class_id': filtered_class_ids,
        'class_name': filtered_class_names
    }

def filter_detections2(detection):
    boxes = detection.xyxy
    confidences = detection.confidence
    class_ids = detection.class_id
    
    num_boxes = len(boxes)
    if num_boxes < 2:
        return detection
    to_keep = [True] * num_boxes

    for i in range(num_boxes):
        if not to_keep[i]:
            continue
        for j in range(i + 1, num_boxes):
            if compute_iou(boxes[i], boxes[j]) > 0.95:
                if confidences[i] < confidences[j]:
                    to_keep[i] = False
                else:
                    to_keep[j] = False

    # Filter the boxes based on the to_keep list
    filtered_boxes = boxes[to_keep]
    filtered_confidences = confidences[to_keep]
    filtered_class_ids = class_ids[to_keep]

    return {
        #Don't need most of this
        #'xyxy': filtered_boxes,
        #'confidence': filtered_confidences,
        #'class_id': filtered_class_ids,
        'class_name': detection['class_name'][to_keep]
    }

In [20]:
def get_folder_name(predictions):
    class_indices = predictions['class_id']
    if not class_indices:
        return "1unknown-deer"
    
    # Get the names using the names mapping
    #class_names = [predictions[0].names[int(class_id)] for class_id in class_indices]
    class_names = predictions['class_name']
    
    class_counts = Counter(class_names)
    #print(class_counts)
    # If there are no class names, use "no_dets"
    if not class_counts:
        return "1unknown-deer"
    
    # Create a list of class names with counts, sorted alphabetically by class name
    folder_parts = [f"{count}{class_name}" for class_name, count in sorted(class_counts.items())]

    # Join parts with underscores
    folder_name = "_".join(folder_parts)

    if any(word in folder_name.lower() for word in ["collar", "eartag", "cliptag"]):
        return "tagged"

    if folder_name == "1fawn_1male-deer":
        folder_name = "1Female deer_1fawn"
    
    return folder_name

def get_folder_name2(predictions):
    class_counts = Counter(predictions['class_name'])
    #print(class_counts)
    # If there are no class names, use "no_dets"
    if not class_counts:
        return "1unknown-deer"
    
    # Create a list of class names with counts, sorted alphabetically by class name
    folder_parts = [f"{count}{class_name}" for class_name, count in sorted(class_counts.items())]

    # Join parts with underscores
    folder_name = "_".join(folder_parts)

    if any(word in folder_name.lower() for word in ["collar", "eartag", "cliptag"]):
        return "tagged"

    if folder_name == "1fawn_1male-deer":
        folder_name = "1Female deer_1fawn"
    
    return folder_name

In [119]:
## Locations
#Grab models, specify where your images are stored and where you want results folders to go
clipmod = YOLO('/Users/heather/runs/detect/yolov8n_earclips/weights/best.pt')
earmod = YOLO('/Users/heather/runs/detect/yolov8n_eartags/weights/best.pt')
collars = YOLO('/Users/heather/runs/detect/yolov8n_collars/weights/best.pt')
deermod = YOLO('/Users/heather/runs/detect/yolov8n_justdeer/weights/best.pt')
#image_folder = '/Users/heather/Desktop/U_Georgia/Chandler_Meetings/heather/CWD_Postdoc/Billionphotos/GR81_07.03.24/MDoutput_07_03_24/animals/deer'
#image_folder = '/Volumes/Unloved/ER25_07.25.23/MDoutput_07_25_23/animals/deer'
#destination_folder = image_folder
#os.makedirs(destination_folder, exist_ok=True)
#initialize empty list:
#results_list = []
main_folder = '/Volumes/Unloved'

In [124]:
# Loop through all subfolders in that main folder
for folder in os.listdir(main_folder):
    # Construct the full path for each subfolder
    subfolder_path = os.path.join(main_folder, folder)
    print(subfolder_path)
    # Check if it's a directory and matches the naming convention (2 letters + 2 digits + date)
    if os.path.isdir(subfolder_path) and len(folder) >= 8 and folder[2:4].isdigit():
        date_part = f'{folder[5:7]}_{folder[8:10]}_{folder[11:13]}'  # Extract date part from the folder name
        # Construct the MDoutput folder name
        md_output_folder = f'MDoutput_{date_part}/animals/deer'
        #print(md_output_folder)

        # Check if the "MDoutput_" folder exists inside the subfolder
        image_folder = os.path.join(main_folder, subfolder_path, md_output_folder)
        destination_folder = image_folder
        os.makedirs(destination_folder, exist_ok=True)
        #initialize empty list:
        results_list = []
        print(image_folder)
        image_files = [filename for filename in os.listdir(image_folder) if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]
        # Get the total number of images
        total_images = len(image_files)
        print(total_images)
        for index, filename in enumerate(image_files, start=1):
        # Check if the file is an image (you might want to adjust the extensions)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                print(f"Processing image {index}/{total_images}")
                # Construct the full path to the image
                image_path = os.path.join(image_folder, filename)
                
                # Process the image
                #First, use earclip model:
                results = clipmod.predict(image_path, conf= .75, verbose = False)
                if results[0].boxes.conf.numel() == 0:
                    #print('no clips')
                    #if no results from clip mod, use ear mod
                    results = earmod.predict(image_path, conf= .5, verbose = False) 
                    if results[0].boxes.conf.numel() == 0:
                        results = collars.predict(image_path, conf= .5, verbose = False) 
                        if results[0].boxes.conf.numel() == 0:
                            results = deermod.predict(image_path, conf= .5, verbose = False) #untagged animals
                            filtered_detection = filter_detections(results)
                            folder_name = get_folder_name(filtered_detection)
                        else: 
                            folder_name = 'tagged' #collars
                    else:
                        folder_name = 'tagged' #ear tags
                else: 
                    folder_name = 'tagged' #ear clips

                #save it:
                # Define the destination path
                dest_folder = os.path.join(destination_folder, folder_name)
                # Create the folder if it doesn't exist
                os.makedirs(dest_folder, exist_ok=True)
                # Define the source and destination paths
                src_path = os.path.join(image_folder, filename)
                dest_path = os.path.join(dest_folder, filename)
                # Move the file
                shutil.move(src_path, dest_path)
                #print(f"Moved {src_path} to {dest_path}")

print("Finished Batch")

/Volumes/Unloved/.Spotlight-V100
/Volumes/Unloved/.fseventsd
/Volumes/Unloved/.Trashes
/Volumes/Unloved/Checked
/Volumes/Unloved/ER59_07.25.23
/Volumes/Unloved/ER59_07.25.23/MDoutput_07_25_23/animals/deer
49
Processing image 1/49
Processing image 2/49
Processing image 3/49
Processing image 4/49
Processing image 5/49
Processing image 6/49
Processing image 7/49
Processing image 8/49
Processing image 9/49
Processing image 10/49
Processing image 11/49
Processing image 12/49
Processing image 13/49
Processing image 14/49
Processing image 15/49
Processing image 16/49
Processing image 17/49
Processing image 18/49
Processing image 19/49
Processing image 20/49
Processing image 21/49
Processing image 22/49
Processing image 23/49
Processing image 24/49
Processing image 25/49
Processing image 26/49
Processing image 27/49
Processing image 28/49
Processing image 29/49
Processing image 30/49
Processing image 31/49
Processing image 32/49
Processing image 33/49
Processing image 34/49
Processing image 35

In [113]:
f'{folder[5:7]}_{folder[8:10]}_{folder[11:13]}'

'07_22_23'